우선 모델링을 먼저 해보자
model list
1. LinearRegression
2. KNN Regressor
3. SVM
4. Ridge, Lasso
5. 

In [3]:
# 1. Load Data
import pandas as pd

dataDF = pd.read_csv('obesity.csv')
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2064 entries, 0 to 2063
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2064 non-null   int64  
 1   Age                             2064 non-null   int64  
 2   Height                          2064 non-null   float64
 3   Weight                          2064 non-null   float64
 4   family_history_with_overweight  2064 non-null   int64  
 5   FAVC                            2064 non-null   int64  
 6   FCVC                            2064 non-null   int64  
 7   NCP                             2064 non-null   int64  
 8   CAEC                            2064 non-null   int64  
 9   SMOKE                           2064 non-null   int64  
 10  CH2O                            2064 non-null   int64  
 11  SCC                             2064 non-null   int64  
 12  FAF                             20

In [5]:
# 2. Feature Selection
# : ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking', 'NObeyesdad', 'BMI']
# 1) Feature 선별:
#       1. 비만도 제거 NObeyesdad
#       2. 과편향 데이터 제거
dataDF.columns
dataDF.drop(['Weight', 'SMOKE','SCC','MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Walking', 'NObeyesdad'], axis=1, inplace=True)
dataDF.columns

Index(['Gender', 'Age', 'Height', 'family_history_with_overweight', 'FAVC',
       'FCVC', 'NCP', 'CAEC', 'CH2O', 'FAF', 'TUE', 'CALC',
       'MTRANS_Automobile', 'MTRANS_Public_Transportation', 'BMI'],
      dtype='object')

In [9]:
# 2) 컬럼명 수정
dataDF.columns = ['성별', '나이', '키', '가족력', '고열량 섭취 빈도', '채소 섭취 빈도', '하루 섭취 끼니 수', '간식 섭취 빈도', '물 섭취량', '활동량', '스마트폰 사용량', '음주 빈도', '이동 수단 : 자가용', '이동수단 : 대중교통', 'BMI']
dataDF

,성별,나이,키,가족력,고열량 섭취 빈도,채소 섭취 빈도,하루 섭취 끼니 수,간식 섭취 빈도,물 섭취량,활동량,스마트폰 사용량,음주 빈도,이동 수단 : 자가용,이동수단 : 대중교통,BMI
0,0,21,1.620,0,0,2,3,2,2,0,1,3,0,1,24.386526
1,0,21,1.520,0,0,3,3,2,3,3,0,2,0,1,24.238227
2,1,23,1.800,0,0,2,3,2,2,2,1,1,0,1,23.765432
3,1,27,1.800,1,0,3,3,2,2,2,0,1,0,0,26.851852
4,1,22,1.780,1,0,2,1,2,2,0,0,2,0,1,28.342381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,0,21,1.711,0,1,3,3,2,2,2,1,2,0,1,44.884392
2060,0,22,1.749,0,1,3,3,2,2,1,1,2,0,1,43.707080
2061,0,23,1.752,0,1,3,3,2,2,1,1,2,0,1,43.557526
2062,0,24,1.739,0,1,3,3,2,3,1,1,2,0,1,44.078924


In [10]:
# 3) 인사이트 : 상관계수, 분포도

In [11]:
# 4) Feature, Target Split
feature = dataDF.drop('BMI', axis=1)
target = dataDF['BMI']
print(feature.shape, target.shape)

# 5) train, test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=11)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2064, 14) (2064,)
(1651, 14) (413, 14) (1651,) (413,)


In [13]:
# 6) Standardize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# 3. Model Selection
# 1) KNN Regression
from sklearn.neighbors import KNeighborsRegressor

knn_model = KNeighborsRegressor()
knn_model.fit(X_train_scaled, y_train)

# 2) Linear Regression
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)

# 3) SVM
from sklearn.svm import SVR

svm_model = SVR()
svm_model.fit(X_train_scaled, y_train)

# 4. Evaluation
print(f'[knn] train : {knn_model.score(X_train_scaled, y_train)}, test : {knn_model.score(X_test_scaled, y_test)}')
print(f'[linear] train : {linear_model.score(X_train_scaled, y_train)}, test : {linear_model.score(X_test_scaled, y_test)}')
print(f'[svm] train : {svm_model.score(X_train_scaled, y_train)}, test : {svm_model.score(X_test_scaled, y_test)}')

[knn] train : 0.872378698194948, test : 0.7849117032495538
[linear] train : 0.45898049219526826, test : 0.4000346467827014
[svm] train : 0.7859196207537547, test : 0.7442667599027977
